In [2]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from numpy.random import seed
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
play_df = pd.read_csv('https://raw.githubusercontent.com/ashfarhangi/Massive_Storage_and_Big_Data/master/data/Shakespeare_data.csv')
play_df
all_lines = [h for h in play_df.PlayerLine]

print(len(all_lines))

111396


In [4]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in all_lines]
corpus[:10]

['act i',
 'scene i london the palace',
 'enter king henry lord john of lancaster the earl of westmoreland sir walter blunt and others',
 'so shaken as we are so wan with care',
 'find we a time for frighted peace to pant',
 'and breathe shortwinded accents of new broils',
 'to be commenced in strands afar remote',
 'no more the thirsty entrance of this soil',
 'shall daub her lips with her own childrens blood',
 'nor more shall trenching war channel her fields']

In [5]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    corpus = corpus[:7000]
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[523, 4],
 [142, 4],
 [142, 4, 339],
 [142, 4, 339, 1],
 [142, 4, 339, 1, 670],
 [53, 41],
 [53, 41, 84],
 [53, 41, 84, 29],
 [53, 41, 84, 29, 124],
 [53, 41, 84, 29, 124, 3]]

In [6]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
predictors.shape, label.shape

((45584, 33), (45584, 6543))

In [7]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(512))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='sigmoid'))

    model.compile(loss='categorical_crossentropy', optimizer='sgd')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 33, 10)            65430     
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1071104   
_________________________________________________________________
dense (Dense)                (None, 6543)              3356559   
Total params: 4,493,093
Trainable params: 4,493,093
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(predictors, label, epochs=2, verbose=1)

Epoch 1/2
1425/1425 [==============================] - 9s 6ms/step - loss: 8.7736
Epoch 2/2
1425/1425 [==============================] - 9s 6ms/step - loss: 8.7394


In [ ]:
model.fit(predictors, label, epochs=20, verbose=2)

Epoch 1/20
1425/1425 - 8s - loss: 8.5448
Epoch 2/20
1425/1425 - 8s - loss: 8.3918
Epoch 3/20
1425/1425 - 8s - loss: 8.2754
Epoch 4/20
1425/1425 - 8s - loss: 8.1367
Epoch 5/20
1425/1425 - 8s - loss: 7.9726
Epoch 6/20
1425/1425 - 8s - loss: 7.8031
Epoch 7/20
1425/1425 - 8s - loss: 7.6475
Epoch 8/20
1425/1425 - 8s - loss: 7.5126
Epoch 9/20
1425/1425 - 8s - loss: 7.3980
Epoch 10/20
1425/1425 - 8s - loss: 7.3014
Epoch 11/20
1425/1425 - 8s - loss: 7.2203
Epoch 12/20
1425/1425 - 8s - loss: 7.1537
Epoch 13/20


In [ ]:
model.fit(predictors, label, epochs=20, verbose=0)

# Generation

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
print ("1. ",generate_text("Julius", 20, model, max_sequence_len))
print ("2. ",generate_text("Thou", 20, model, max_sequence_len))
print ("3. ",generate_text("King is", 20, model, max_sequence_len))
print ("4. ",generate_text("Death of", 20, model, max_sequence_len))
print ("5. ",generate_text("The Princess", 20, model, max_sequence_len))
print ("6. ",generate_text("Thanos", 20, model, max_sequence_len))